# Env Setting

In [1]:
import sys 
# sys.path.clear()
sys.path.insert(0, 'D:\\Anaconda\\envs\\tensorflow_cpu\\python36.zip')
sys.path.insert(0, 'D:\\Anaconda\\envs\\tensorflow_cpu\\DLLs')
sys.path.insert(0, 'D:\\Anaconda\\envs\\tensorflow_cpu\\lib')
sys.path.insert(0, 'D:\\Anaconda\\envs\\tensorflow_cpu')
sys.path.insert(0, 'D:\\Anaconda\\envs\\tensorflow_cpu\\lib\\site-packages')
sys.path.insert(0, '')

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.0.0


In [3]:
from scipy.sparse import csr_matrix, load_npz, save_npz
from tqdm import tqdm
from sklearn.preprocessing import normalize
import datetime
import json
import pandas as pd
import time
import yaml
import scipy.sparse as sparse
from ast import literal_eval

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import normalize
from sklearn.linear_model import Ridge

# Loading data

In [5]:
# Load Original Data
df_train = pd.read_csv('../../data/yelp/Train.csv',encoding='latin-1')
df_valid = pd.read_csv('../../data/yelp/Valid.csv',encoding='latin-1')
df_test = pd.read_csv('../../data/yelp/Test.csv',encoding='latin-1')
keyphrases = pd.read_csv('../../data/yelp/KeyPhrases.csv')['Phrases'].tolist()
keyphrase_popularity = np.loadtxt('../data/yelp/'+'keyphrase_popularity.txt', dtype=int)

In [6]:
# Load U-I Data 
rtrain = load_npz("../../data/yelp/Rtrain.npz")
rvalid = load_npz("../../data/yelp/Rvalid.npz")
rtest = load_npz("../../data/yelp/Rtest.npz")

In [7]:
# Load user/item keyphrase data
U_K = load_npz("../../data/yelp/U_K.npz")
I_K = load_npz("../../data/yelp/I_K.npz")

# Forward Models

In [8]:
# Models
from sklearn.metrics.pairwise import cosine_similarity
def train(matrix_train):
    similarity = cosine_similarity(X=matrix_train, Y=None, dense_output=True)
    return similarity

def get_I_K(df, row_name = 'ItemIndex', shape = (3668,75)):
    rows = []
    cols = []
    vals = []
    for i in tqdm(range(df.shape[0])):
        key_vector = literal_eval(df['keyVector'][i])
        rows.extend([df[row_name][i]]*len(key_vector)) ## Item index
        cols.extend(key_vector) ## Keyword Index
        vals.extend(np.array([1]*len(key_vector)))
    return csr_matrix((vals, (rows, cols)), shape=shape)



def prediction(prediction_score, topK, matrix_Train):

    prediction = []

    for user_index in tqdm(range(matrix_Train.shape[0])):
        vector_u = prediction_score[user_index]
        vector_train = matrix_Train[user_index]
        if len(vector_train.nonzero()[0]) > 0:
            vector_predict = sub_routine(vector_u, vector_train, topK=topK)
        else:
            vector_predict = np.zeros(topK, dtype=np.float32)

        prediction.append(vector_predict)

    return np.vstack(prediction)


def sub_routine(vector_u, vector_train, topK=500):

    train_index = vector_train.nonzero()[1]

    vector_u = vector_u

    candidate_index = np.argpartition(-vector_u, topK+len(train_index))[:topK+len(train_index)]
    vector_u = candidate_index[vector_u[candidate_index].argsort()[::-1]]
    vector_u = np.delete(vector_u, np.isin(vector_u, train_index).nonzero()[0])

    return vector_u[:topK]


In [9]:
def predict(matrix_train, k, similarity, item_similarity_en = False):
    """
    res = similarity * matrix_train    if item_similarity_en = False
    res = similarity * matrix_train.T  if item_similarity_en = True
    """
    prediction_scores = []
    
    if item_similarity_en:
        matrix_train = matrix_train.transpose()
        
    for user_index in tqdm(range(matrix_train.shape[0])):
        # Get user u's prediction scores to all users
        vector_u = similarity[user_index]

        # Get closest K neighbors excluding user u self
        similar_users = vector_u.argsort()[::-1][1:k+1]
        # Get neighbors similarity weights and ratings
        similar_users_weights = similarity[user_index][similar_users]
        similar_users_ratings = matrix_train[similar_users].toarray()

        prediction_scores_u = similar_users_ratings * similar_users_weights[:, np.newaxis]

        prediction_scores.append(np.sum(prediction_scores_u, axis=0))
    res = np.array(prediction_scores)
    
    if item_similarity_en:
        res = res.transpose()
    
    return res

def predict_vector(user_index, matrix_train, k, similarity):
    """
    res = similarity * matrix_train    if item_similarity_en = False
    res = similarity * matrix_train.T  if item_similarity_en = True
    get only user_index row
    """
    vector_u = similarity[user_index]
    
    # Get closest K neighbors excluding user u self
    similar_users = vector_u.argsort()[::-1][1:k+1]
    # Get neighbors similarity weights and ratings
    similar_users_weights = similarity[user_index][similar_users]
    similar_users_ratings = matrix_train[similar_users].toarray()
    prediction_scores_u = similar_users_ratings * similar_users_weights[:, np.newaxis]
    
    return np.sum(prediction_scores_u, axis=0)
    

# Eval Models

In [10]:
# Evluation 
def recallk(vector_true_dense, hits, **unused):
    hits = len(hits.nonzero()[0])
    return float(hits)/len(vector_true_dense)

def precisionk(vector_predict, hits, **unused):
    hits = len(hits.nonzero()[0])
    return float(hits)/len(vector_predict)


def average_precisionk(vector_predict, hits, **unused):
    precisions = np.cumsum(hits, dtype=np.float32)/range(1, len(vector_predict)+1)
    return np.mean(precisions)


def r_precision(vector_true_dense, vector_predict, **unused):
    vector_predict_short = vector_predict[:len(vector_true_dense)]
    hits = len(np.isin(vector_predict_short, vector_true_dense).nonzero()[0])
    return float(hits)/len(vector_true_dense)


def _dcg_support(size):
    arr = np.arange(1, size+1)+1
    return 1./np.log2(arr)


def ndcg(vector_true_dense, vector_predict, hits):
    idcg = np.sum(_dcg_support(len(vector_true_dense)))
    dcg_base = _dcg_support(len(vector_predict))
    dcg_base[np.logical_not(hits)] = 0
    dcg = np.sum(dcg_base)
    return dcg/idcg


def click(hits, **unused):
    first_hit = next((i for i, x in enumerate(hits) if x), None)
    if first_hit is None:
        return 5
    else:
        return first_hit/10


def evaluate(matrix_Predict, matrix_Test, metric_names =['R-Precision', 'NDCG', 'Precision', 'Recall', 'MAP'], atK = [5, 10, 15, 20, 50], analytical=False):
    """
    :param matrix_U: Latent representations of users, for LRecs it is RQ, for ALSs it is U
    :param matrix_V: Latent representations of items, for LRecs it is Q, for ALSs it is V
    :param matrix_Train: Rating matrix for training, features.
    :param matrix_Test: Rating matrix for evaluation, true labels.
    :param k: Top K retrieval
    :param metric_names: Evaluation metrics
    :return:
    """
    global_metrics = {
        "R-Precision": r_precision,
        "NDCG": ndcg,
        "Clicks": click
    }

    local_metrics = {
        "Precision": precisionk,
        "Recall": recallk,
        "MAP": average_precisionk
    }

    output = dict()

    num_users = matrix_Predict.shape[0]

    for k in atK:

        local_metric_names = list(set(metric_names).intersection(local_metrics.keys()))
        results = {name: [] for name in local_metric_names}
        topK_Predict = matrix_Predict[:, :k]

        for user_index in tqdm(range(topK_Predict.shape[0])):
            vector_predict = topK_Predict[user_index]
            if len(vector_predict.nonzero()[0]) > 0:
                vector_true = matrix_Test[user_index]
                vector_true_dense = vector_true.nonzero()[1]
                hits = np.isin(vector_predict, vector_true_dense)

                if vector_true_dense.size > 0:
                    for name in local_metric_names:
                        results[name].append(local_metrics[name](vector_true_dense=vector_true_dense,
                                                                 vector_predict=vector_predict,
                                                                 hits=hits))

        results_summary = dict()
        if analytical:
            for name in local_metric_names:
                results_summary['{0}@{1}'.format(name, k)] = results[name]
        else:
            for name in local_metric_names:
                results_summary['{0}@{1}'.format(name, k)] = (np.average(results[name]),
                                                              1.96*np.std(results[name])/np.sqrt(num_users))
        output.update(results_summary)

    global_metric_names = list(set(metric_names).intersection(global_metrics.keys()))
    results = {name: [] for name in global_metric_names}

    topK_Predict = matrix_Predict[:]

    for user_index in tqdm(range(topK_Predict.shape[0])):
        vector_predict = topK_Predict[user_index]

        if len(vector_predict.nonzero()[0]) > 0:
            vector_true = matrix_Test[user_index]
            vector_true_dense = vector_true.nonzero()[1]
            hits = np.isin(vector_predict, vector_true_dense)

            # if user_index == 1:
            #     import ipdb;
            #     ipdb.set_trace()

            if vector_true_dense.size > 0:
                for name in global_metric_names:
                    results[name].append(global_metrics[name](vector_true_dense=vector_true_dense,
                                                              vector_predict=vector_predict,
                                                              hits=hits))

    results_summary = dict()
    if analytical:
        for name in global_metric_names:
            results_summary[name] = results[name]
    else:
        for name in global_metric_names:
            results_summary[name] = (np.average(results[name]), 1.96*np.std(results[name])/np.sqrt(num_users))
    output.update(results_summary)

    return output



# Forward Initial Prediction

## User-Item KNN

In [11]:
similarity = normalize(train(rtrain))
user_item_prediction_score = predict(rtrain, 100, similarity, item_similarity_en= False)
user_item_predict = prediction(user_item_prediction_score, 50, rtrain)
user_item_res = evaluate(user_item_predict, rtest)

100%|████████████████████████████████████████████████████████████████████████████| 2343/2343 [00:00<00:00, 4166.57it/s]


In [12]:
# k = 100 
user_item_res

{'MAP@10': (0.06333952750429245, 0.00455145183026277),
 'MAP@15': (0.05872249612550844, 0.0038121823597348156),
 'MAP@20': (0.055196280875748356, 0.003345258771111864),
 'MAP@5': (0.06940666362391602, 0.0060646277121995185),
 'MAP@50': (0.04436838784245958, 0.0022020570312340938),
 'NDCG': (0.09071795198195, 0.003803970590016347),
 'Precision@10': (0.05330899132816066, 0.0032544740534870857),
 'Precision@15': (0.04698006998326487, 0.0025622684368576416),
 'Precision@20': (0.043336376083979916, 0.002222006808632301),
 'Precision@5': (0.06462802373345505, 0.004754606217931856),
 'Precision@50': (0.032889091738931994, 0.0014317314500480152),
 'R-Precision': (0.048464138894968055, 0.0027869069242192506),
 'Recall@10': (0.04269615369598775, 0.0027831077562657665),
 'Recall@15': (0.05562887733868642, 0.0031389991916236284),
 'Recall@20': (0.0677664821917642, 0.003424484045821138),
 'Recall@5': (0.026408137823500974, 0.002191309117794643),
 'Recall@50': (0.12696642809611336, 0.004824188397130

## Latent Similarity Matrix Learned with Linear Regression¶

In [11]:
# Training 
X = normalize(U_K.todense())
y = normalize(train(rtrain))
clf = Ridge(alpha=0.1).fit(X, y) # Optimality at L2 regularization = 0.1
lr_similarity = clf.predict(np.array(X))

In [ ]:
# Prediciting
similarity = lr_similarity
lr_prediction_score = predict(rtrain, 100, similarity, item_similarity_en= False)
lr_predict = prediction(lr_prediction_score, 50, rtrain)
lr_res = evaluate(lr_predict, rtest)

In [ ]:
# k = 100
lr_res

# Critiquing 

In [12]:
# One hot encoding of critiquing
def get_critiqued_UK(user_keyphrase_frequency,user_index,critiqued_keyphrase):
    """
    user_keyphrase_frequency is the U_K matrix (csr sparse matrix)
    return the one-hot encoding of the critique
    """
    U_K_cp = user_keyphrase_frequency.copy()
    U_K_cp[user_index] = 0
    U_K_cp[user_index,critiqued_keyphrase] = 1
    return U_K_cp

def project_one_hot_encoding(reg, user_keyphrase_frequency,user_index = 0,critiqued_keyphrase = 0, normalize_en = True):
    """
    Return the projection on user_sim space from one-hot encoding of critiqued keyphrase
    The res[user_index] should be target embedding row
    """
    critiqued_matrix = get_critiqued_UK(user_keyphrase_frequency, user_index, critiqued_keyphrase)
    res = reg.predict(critiqued_matrix)
    if normalize_en:
        res = normalize((res))
    return res

In [13]:
def get_initial_predictions(X = normalize(U_K.todense()), y = normalize(train(rtrain)),
                            matrix_Train = rtrain, k = 100):
    clf = Ridge(alpha=0.1).fit(X, y)
    similarity = normalize(train(matrix_Train))
    user_item_prediction_score = predict(matrix_Train, k, similarity, item_similarity_en= False)
    return user_item_prediction_score, clf
def get_valid_keyphrases(keyphrase_freq,top_recommendations,item = None,threshold=50,mutiple_keyphrases_en = False, top_items = None):
    """
    Wrapper function to get either top 1 or top n keyphrases
    """
    if mutiple_keyphrases_en:
        top_keyphrases = []
        for item in top_items:
            top_keyphrases.extend(get_valid_keyphrases_for_one_item(keyphrase_freq,top_recommendations,item,threshold=threshold))
        return np.ravel(list(set(top_keyphrases))) # remove duplicate and reformat to np array
    else:
        return get_valid_keyphrases_for_one_item(keyphrase_freq,top_recommendations,item,threshold=threshold)

def get_valid_keyphrases_for_one_item(keyphrase_freq,top_recommendations, item,threshold=50):
    """
    Get keyphrases of item that make sense
    E.g. if the item has fewer than threshold=50 keyphrases, get all of them
    otherwise get top 50 keyphrases
    """
    keyphrase_length = len(keyphrase_freq[item].nonzero()[1])
    if keyphrase_length<threshold:
        return keyphrase_freq[item].nonzero()[1]
    else:
        keyphrases = np.ravel(keyphrase_freq[top_recommendations[0]].todense())
        top_keyphrases = np.argsort(keyphrases)[::-1][:threshold]
        return top_keyphrases
    
def predict_vector(user_index, matrix_train, k, similarity, with_keyphrase = False, 
                   keyphrase_freq = None, critiqued_keyphrase = None, alpha = 0):
    """
    get only user_index row
    if with_keyphrase = True, then penalize items without critiqued_keyphrase to alpha (default = 0)
    """
    vector_u = similarity[user_index]
    
    # Get closest K neighbors excluding user u self
    similar_users = vector_u.argsort()[::-1][1:k+1]
    # Get neighbors similarity weights and ratings
    similar_users_weights = similarity[user_index][similar_users]
    similar_users_ratings = matrix_train[similar_users].toarray()
    
    prediction_scores_u = similar_users_ratings * similar_users_weights[:, np.newaxis]
    
    if with_keyphrase == False:
        return np.sum(prediction_scores_u, axis=0)
    
    # Only Predict items with critiqued_keyphrase 
    else:
        prediction_scores = np.sum(prediction_scores_u, axis=0)
#         print (prediction_scores)
        #penalize items without critiqued keyphrase
        items_with_keyphrase = np.ravel(keyphrase_freq.T[critiqued_keyphrase].nonzero()[1])
#         print (items_with_keyphrase)
        #Return the unique values in ar1 that are not in ar2.
        items_without_keyphrase = np.setdiff1d(np.arange(matrix_train.shape[1]), items_with_keyphrase)
        prediction_scores[items_without_keyphrase] = alpha # penalize
        return prediction_scores
#         print (prediction_scores)
#         return prediction_scores/sum(prediction_scores)

    
def get_initial_prediction(user,X = normalize(U_K.todense()), y = normalize(train(rtrain)),
                            matrix_Train = rtrain, k = 100):
    """
    Get the initial knn predictions before critiquing pipelines
    get the linear regression model for critiquing embedding (W_2)
    get the initial user similarity matrix 
    k here is the parameter for KNN
    """
    clf = Ridge(alpha=0.1).fit(X, y)
    similarity = normalize(train(matrix_Train))
    user_item_prediction_score = predict_vector(user, matrix_Train, k, similarity)
    return user_item_prediction_score, clf



In [14]:
# For keyphrase selecting method # 3 "diff" 
def get_item_keyphrase_freq(keyphrase_freq,item):
    """
    Get item's keyphrase frequency 
    """
    count = keyphrase_freq[item].todense()
    return count/np.sum(count)

In [15]:
# Utility function for getting restaurant info from ItemIndex
def get_business_df(path = "../../data/yelp/business.json" ):
    with open(path,encoding="utf8") as json_file:
        data = json_file.readlines()
        data = list(map(json.loads, data))
    df = pd.DataFrame(data)
    
    return df

def get_restaurant_info(business_df, business_id, name = True, review_count = True, stars = True ):
    output_list = {}
    row_idx = int(business_df.index[business_df['business_id'] == business_id].tolist()[0])
    if name == True:
        output_list['name'] = business_df['name'][row_idx].encode('utf-8').strip()
    if review_count == True:
        output_list['review_count'] = business_df['review_count'][row_idx]
    if stars == True:
        output_list['stars'] = business_df['stars'][row_idx] 
    return output_list

# def get_businessid_from_Itemindex(ItemIndex_list, itemindex):
#     return ItemIndex_list['business_id'].tolist()[itemindex]

def get_restaurant_name(df_train, business_df, ItemIndex):
    rows = np.where(df_train['ItemIndex'] == ItemIndex)
    if len(rows)!= 0:
        business_id = df_train.loc[rows[0][0]]['business_id']
        item_info = get_restaurant_info(business_df, business_id)
        return item_info['name']
    return "NOT_FOUND"

In [16]:
def get_keyphrase_popularity(df,keyphrases):
    """
    Get keyphrase popularity (count) from dataframe
    """
    keyphrase_popularity = np.zeros(len(keyphrases)) #initialize
    for i in range(len(df)):
        keyphrase_vector = literal_eval(df['keyVector'][i])
        keyphrase_popularity[keyphrase_vector] += 1 # count
    return keyphrase_popularity

In [17]:
# keyphrase_popularity = get_keyphrase_popularity(df_train,keyphrases)

# Save and load
# np.savetxt('../data/yelp/'+'keyphrase_popularity.txt', keyphrase_popularity, fmt='%d')
keyphrase_popularity = np.loadtxt('../data/yelp/'+'keyphrase_popularity.txt', dtype=int)

In [18]:
business_df = get_business_df()

In [99]:
business_df.to_csv('../../data/yelp/business_df.csv')

In [19]:
# Initialize df for storing the experiment

# post_ranki is post rank with different lambda ratio for combining pre-post User similarity matrix 
columns = ['user_id', 'target_item', 'item_name', 'iter', 'pre_rank', 
           'top_prediction_item_name','critiqued_keyphrase', 'keyphrase_name', 
           'post_rank0', 
           'post_rank1', 
           'post_rank2', 
           'post_rank3', 
           'post_rank4', 
           'post_rank5', 
           'post_rank6', 
           'post_rank7', 
           'post_rank8',
           'post_rank9',
           'post_rank10',
           'num_existing_keyphrases'] 
df = pd.DataFrame(columns=columns)
row = {}

## Experiment Pipeline

In [20]:
def single_step_critiquing_experiment_copy(user = 2, 
                           keyphrase_length_threshold = 150, 
                           max_iteration_threshold = 5,
                           k = 50,
                           df = df,
                           row = row,
                           business_df = business_df,
                           keyphrases = keyphrases,
                           keyphrase_popularity = keyphrase_popularity, 
                           keyphrase_selection_method = 'random',
                           recommend_type = 'all',
                           lams = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1] 
                          ):
    """
    k: HR@k 
    keyphrase_length_threshold: limit the number of keyphrases in top recommended item
    keyphrase_selection_method: 'random': randomly select keyphrase from wanted_keyphrases
                                'pop': always select the most popular keyphrase in wanted_keyphrases
                                'diff': select the keyphrase with largest frequency difference between top recommended 
                                        item and target item.
    recommend_type: 'all': recommend all items
                    'upper' (only_with_critiqued_keyphrase): recommend items with only critiqued_keyphrase
    lam: modified_matrix = lam*origianl_matrix + (1-lam)*critiquing_embedding 
    """
    
    row['user_id'] = user
    print ('User ID ', user)
    
    # Set up (move to header line later)
    matrix_Train = rtrain
    matrix_Test = rtest
    keyphrase_freq = I_K
    num_items = rtrain.shape[1]
    max_wanted_keyphrase = 10 # for keyphrase_selection_method == "diff"
    initial_user_similarity_embedding = normalize(train(matrix_Train))
    
    # Get wanted items 
    candidate_items = matrix_Test[user].nonzero()[1]
    train_items = matrix_Train[user].nonzero()[1]
    wanted_items = np.setdiff1d(candidate_items, train_items)
    print ('wanted_items length: ',len(wanted_items))
    
    # Get initial forward prediction 
    prediction_score,clf = get_initial_prediction(user, X = normalize(U_K.todense()), y = normalize(train(rtrain)),
                            matrix_Train = rtrain, k = 100)
    
    # Get initial top recommended item(s)
    top_recommendations = np.argsort(prediction_score)[::-1]
    print ("Initial top recommendation index",top_recommendations[0])
    try:
        row['top_prediction_item_name'] = get_restaurant_name(df_train, business_df, top_recommendations[0])
    # in case we cannot get the restaurant name
    except: 
        row['top_prediction_item_name'] = 'CANNOT_FIND'
        print ('Cannot get restaurant name for ItemIndex: ', top_recommendations[0])
    
    
    # Get top recommended item's keyphrases
    top_item = top_recommendations[0] 
    top_recommend_keyphrases = get_valid_keyphrases(keyphrase_freq,
                                                    top_recommendations, 
                                                    item = top_item,
                                                    threshold=keyphrase_length_threshold,
                                                    mutiple_keyphrases_en = False, 
                                                    top_items = None)
    print ('num_top_recommended_keyphrases ',len(top_recommend_keyphrases))

    if keyphrase_selection_method == 'diff':
        top_recommended_keyphrase_freq = get_item_keyphrase_freq(keyphrase_freq,item = top_item)
    
    
    #####################################
    # For each item, do the critiquing
    
    #limit the item to only 10
    num_target_item = 0 # initialize item count
    
    for item in wanted_items:    
        print ('target_item: ', item)
        row['target_item'] = item
        try:
            row['item_name'] = get_restaurant_name(df_train, business_df, item)
        except:
            row['item_name'] = 'CANNOT_FIND'
            print ('Cannot get restaurant name for ItemIndex: ', item)

        # Get pre-critiquing rank
        initial_rank = np.where(item == np.argsort(prediction_score)[::-1])[0][0]
#         print ('target_item initial rank', int(initial_rank))
        row['pre_rank'] = int(initial_rank)

        # Get the target item's existing keyphrases
        item_keyphrases = keyphrase_freq[item].nonzero()[1]
#         print ('num_existing_keyphrases ',len(item_keyphrases))
        
        if keyphrase_selection_method == 'diff':
            target_keyphrase_freq = get_item_keyphrase_freq(keyphrase_freq,item = item)
            # indicate the keyphrase with large freq in target_item but small_keyphrase in top_recommended items
            diff_keyphrase_freq = target_keyphrase_freq - top_recommended_keyphrase_freq
            
        # Get wanted keyphrases
        if keyphrase_selection_method != 'diff':
            # Get keyphrases that is not in the top recommended items but in the target item (we can select)
            wanted_keyphrases = np.setdiff1d(item_keyphrases,top_recommend_keyphrases)

            if len(wanted_keyphrases) == 0:
                print ("wanted_keyphrases is empty")
                break
            row['num_existing_keyphrases'] = len(wanted_keyphrases)
            
        # For 'diff'
        else:
            wanted_keyphrases = np.argsort(np.ravel(diff_keyphrase_freq))[::-1][:max_wanted_keyphrase]
            row['num_existing_keyphrases'] = len(wanted_keyphrases)

        affected_items = np.array([])
        modified_matrix = initial_user_similarity_embedding # initialize user similarity embedding
        
        #############################################
        # Critiquing iteration
        for iteration in range(max_iteration_threshold):
            print ('cur_iter ', iteration)
            row['iter'] = iteration
            if keyphrase_selection_method == 'random':
                # Randomly critique one keyphrase
                critiqued_keyphrase = np.random.choice(wanted_keyphrases, size=1, replace=False)[0]
            elif keyphrase_selection_method == 'pop':
                # Always critique the most popular keyphrase
                critiqued_keyphrase = wanted_keyphrases[np.argmax(keyphrase_popularity[wanted_keyphrases])]
            elif keyphrase_selection_method == 'diff':
                # critique the keyphrase with largest freq diff between top recommended_item and target_item
                critiqued_keyphrase = wanted_keyphrases[0]
#                 print (critiqued_keyphrase)
            
#             print ('critiqued_keyphrase ,',critiqued_keyphrase, keyphrases[critiqued_keyphrase])
            row['critiqued_keyphrase'] = critiqued_keyphrase
            row['keyphrase_name'] = keyphrases[critiqued_keyphrase]
            
            # Do not critique this keyphrase next time
            wanted_keyphrases = np.delete(wanted_keyphrases, np.where(critiqued_keyphrase == wanted_keyphrases))
            if len(wanted_keyphrases) == 0: 
                print ('no more keyphrase available')
                break
            
            # Get affected items (items have critiqued keyphrase)
            current_affected_items = keyphrase_freq[:, critiqued_keyphrase].nonzero()[0]
            affected_items = np.unique(np.concatenate((affected_items, current_affected_items))).astype(int) 
            unaffected_items = np.setdiff1d(range(num_items), affected_items)

            # Critiquing Embedding

            # One hot encoding
            critiqued_matrix_onehot = get_critiqued_UK(U_K,user,critiqued_keyphrase)
            critiqued_matrix = clf.predict(critiqued_matrix_onehot)
            critiqued_matrix = normalize(critiqued_matrix)
            
#             critiqued_matrix = project_one_hot_encoding(clf, 
#                                                         U_K,
#                                                         user_index = user,
#                                                         critiqued_keyphrase = critiqued_keyphrase, 
#                                                         normalize_en = True)

#             modified_matrix = modified_matrix + critiqued_matrix # averaging user-item embedding and critiquing embeeding
            
            # Warning!!! The following is used only for testing single step critiquing, 
            # for full average critiquing, use the above commented line 
            post_ranks = []
            for lam in lams:
                modified_matrix = (1-lam)*normalize(train(matrix_Train)) + lam*critiqued_matrix 
#                 modified_matrix = normalize(modified_matrix)
                if lam == 0:
                    print (modified_matrix == initial_user_similarity_embedding)
            
                # Get new predictions from modified embedding
                if recommend_type == 'all':
                    prediction_scores_u = predict_vector(user, matrix_Train, 100, modified_matrix)
                if recommend_type == 'upper':
                    prediction_scores_u = predict_vector(user, matrix_Train, 100, modified_matrix, 
                                                         with_keyphrase = True, 
                                                         keyphrase_freq = keyphrase_freq, 
                                                         critiqued_keyphrase = critiqued_keyphrase, 
                                                         alpha = 0)
                post_critique_rank = np.where(item == np.argsort(prediction_scores_u)[::-1])[0][0]
                print ('target_item post-critique rank with lambda '+str(lam), int(post_critique_rank))
                post_rank = int(post_critique_rank)
                post_ranks.append(post_rank)
            row['post_rank'] = post_ranks
            row['post_rank0'] = post_ranks[0]
            row['post_rank1'] = post_ranks[1]
            row['post_rank2'] = post_ranks[2]
            row['post_rank3'] = post_ranks[3]
            row['post_rank4'] = post_ranks[4]
            row['post_rank5'] = post_ranks[5]
            row['post_rank6'] = post_ranks[6]
            row['post_rank7'] = post_ranks[7]
            row['post_rank8'] = post_ranks[8]
            row['post_rank9'] = post_ranks[9]
            row['post_rank10'] = post_ranks[10]
            df = df.append(row, ignore_index=True)
            

        # break after got 10 target items 
        num_target_item += 1
        if num_target_item >10: # only want max 10 items per user
            break
            
    return df

In [37]:
def single_step_critiquing_experiment(user = 2, 
                           keyphrase_length_threshold = 150, 
                           max_iteration_threshold = 5,
                           k = 50,
                           df = df,
                           row = row,
                           business_df = business_df,
                           keyphrases = keyphrases,
                           keyphrase_popularity = keyphrase_popularity, 
                           keyphrase_selection_method = 'random',
                           recommend_type = 'all',
                           lams = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1] 
                          ):
    """
    k: HR@k 
    keyphrase_length_threshold: limit the number of keyphrases in top recommended item
    keyphrase_selection_method: 'random': randomly select keyphrase from wanted_keyphrases
                                'pop': always select the most popular keyphrase in wanted_keyphrases
                                'diff': select the keyphrase with largest frequency difference between top recommended 
                                        item and target item.
    recommend_type: 'all': recommend all items
                    'upper' (only_with_critiqued_keyphrase): recommend items with only critiqued_keyphrase
    lam: modified_matrix = lam*origianl_matrix + (1-lam)*critiquing_embedding 
    """
    
    row['user_id'] = user
    print ('User ID ', user)
    
    # Set up (move to header line later)
    matrix_Train = rtrain
    matrix_Test = rtest
    keyphrase_freq = I_K
    num_items = rtrain.shape[1]
    max_wanted_keyphrase = 10 # for keyphrase_selection_method == "diff"
    initial_user_similarity_embedding = normalize(train(matrix_Train))
    
    # Get wanted items 
    candidate_items = matrix_Test[user].nonzero()[1]
    train_items = matrix_Train[user].nonzero()[1]
    wanted_items = np.setdiff1d(candidate_items, train_items)
    print ('wanted_items length: ',len(wanted_items))
    
    # Get initial forward prediction 
    prediction_score,clf = get_initial_prediction(user, X = normalize(U_K.todense()), y = normalize(train(rtrain)),
                            matrix_Train = rtrain, k = 100)
    
    # Get initial top recommended item(s)
    top_recommendations = np.argsort(prediction_score)[::-1]
    print ("Initial top recommendation index",top_recommendations[0])
    try:
        row['top_prediction_item_name'] = get_restaurant_name(df_train, business_df, top_recommendations[0])
    # in case we cannot get the restaurant name
    except: 
        row['top_prediction_item_name'] = 'CANNOT_FIND'
        print ('Cannot get restaurant name for ItemIndex: ', top_recommendations[0])
    
    
    # Get top recommended item's keyphrases
    top_item = top_recommendations[0] 
    top_recommend_keyphrases = get_valid_keyphrases(keyphrase_freq,
                                                    top_recommendations, 
                                                    item = top_item,
                                                    threshold=keyphrase_length_threshold,
                                                    mutiple_keyphrases_en = False, 
                                                    top_items = None)
    top_recommended_keyphrase_freq = get_item_keyphrase_freq(keyphrase_freq,item = top_item)
    
    
    #####################################
    # For each item, do the critiquing
    
    #limit the item to only 10
    num_target_item = 0 # initialize item count
    
    for item in wanted_items:    
        print ('target_item: ', item)
        row['target_item'] = item
        try:
            row['item_name'] = get_restaurant_name(df_train, business_df, item)
        except:
            row['item_name'] = 'CANNOT_FIND'
            print ('Cannot get restaurant name for ItemIndex: ', item)

        # Get pre-critiquing rank
        initial_rank = np.where(item == np.argsort(prediction_score)[::-1])[0][0]
#         print ('target_item initial rank', int(initial_rank))
        row['pre_rank'] = int(initial_rank)

        # Get the target item's existing keyphrases
        item_keyphrases = keyphrase_freq[item].nonzero()[1]
#         print ('num_existing_keyphrases ',len(item_keyphrases))
        
#         if keyphrase_selection_method == 'diff':
#             target_keyphrase_freq = get_item_keyphrase_freq(keyphrase_freq,item = item)
#             # indicate the keyphrase with large freq in target_item but small_keyphrase in top_recommended items
#             diff_keyphrase_freq = target_keyphrase_freq - top_recommended_keyphrase_freq
            
#         # Get wanted keyphrases
#         if keyphrase_selection_method != 'diff':
#             # Get keyphrases that is not in the top recommended items but in the target item (we can select)
#             wanted_keyphrases = np.setdiff1d(item_keyphrases,top_recommend_keyphrases)

#             if len(wanted_keyphrases) == 0:
#                 print ("wanted_keyphrases is empty")
#                 break
#             row['num_existing_keyphrases'] = len(wanted_keyphrases)
            
#         # For 'diff'
#         else:
#             wanted_keyphrases = np.argsort(np.ravel(diff_keyphrase_freq))[::-1][:max_wanted_keyphrase]
#             row['num_existing_keyphrases'] = len(wanted_keyphrases)
        
        # For diff 
        target_keyphrase_freq = get_item_keyphrase_freq(keyphrase_freq,item = item)
        diff_keyphrase_freq = target_keyphrase_freq - top_recommended_keyphrase_freq
        
        wanted_keyphrases_random = np.setdiff1d(item_keyphrases,top_recommend_keyphrases)
        wanted_keyphrases_pop = np.setdiff1d(item_keyphrases,top_recommend_keyphrases)
        wanted_keyphrases_diff = np.argsort(np.ravel(diff_keyphrase_freq))[::-1][:max_wanted_keyphrase]
        
            
        affected_items = np.array([])
        modified_matrix = initial_user_similarity_embedding # initialize user similarity embedding
        
        #############################################
        # Critiquing iteration
        for iteration in range(max_iteration_threshold):
            print ('cur_iter ', iteration)
            row['iter'] = iteration
            
#             if keyphrase_selection_method == 'random':
#                 # Randomly critique one keyphrase
#                 critiqued_keyphrase = np.random.choice(wanted_keyphrases, size=1, replace=False)[0]
#             elif keyphrase_selection_method == 'pop':
#                 # Always critique the most popular keyphrase
#                 critiqued_keyphrase = wanted_keyphrases[np.argmax(keyphrase_popularity[wanted_keyphrases])]
#             elif keyphrase_selection_method == 'diff':
#                 # critique the keyphrase with largest freq diff between top recommended_item and target_item
#                 critiqued_keyphrase = wanted_keyphrases[0]
            print (wanted_keyphrases_random)
            if len(wanted_keyphrases_random) == 0 or len(wanted_keyphrases_diff) == 0: 
                print ('no more keyphrase available')
                break
            critiqued_keyphrase_random = np.random.choice(wanted_keyphrases_random, size=1, replace=False)[0]
            critiqued_keyphrase_pop = wanted_keyphrases_pop[np.argmin(keyphrase_popularity[wanted_keyphrases_pop])] # Select the least popular
            critiqued_keyphrase_diff = wanted_keyphrases_diff[0]
            
            row['critiqued_keyphrase_random'] = critiqued_keyphrase_random
            row['keyphrase_name_random'] = keyphrases[critiqued_keyphrase_random]
            row['critiqued_keyphrase_pop'] = critiqued_keyphrase_pop
            row['keyphrase_name_pop'] = keyphrases[critiqued_keyphrase_pop]
            row['critiqued_keyphrase_diff'] = critiqued_keyphrase_diff
            row['keyphrase_name_diff'] = keyphrases[critiqued_keyphrase_diff]
            
            # Do not critique this keyphrase next time
            wanted_keyphrases_random = np.delete(wanted_keyphrases_random, np.where(critiqued_keyphrase_random == wanted_keyphrases_random))
            wanted_keyphrases_pop = np.delete(wanted_keyphrases_pop, np.where(critiqued_keyphrase_pop == wanted_keyphrases_pop))
            wanted_keyphrases_diff = np.delete(wanted_keyphrases_diff, np.where(critiqued_keyphrase_diff == wanted_keyphrases_diff))
            
            
            # Get affected items (items have critiqued keyphrase)
#             current_affected_items = keyphrase_freq[:, critiqued_keyphrase].nonzero()[0]
#             affected_items = np.unique(np.concatenate((affected_items, current_affected_items))).astype(int) 
#             unaffected_items = np.setdiff1d(range(num_items), affected_items)

            # Critiquing Embedding

            # One hot encoding
            critiqued_matrix_onehot = get_critiqued_UK(U_K,user,critiqued_keyphrase_random)
            critiqued_matrix = clf.predict(critiqued_matrix_onehot)
            critiqued_matrix_random = normalize(critiqued_matrix)

            critiqued_matrix_onehot = get_critiqued_UK(U_K,user,critiqued_keyphrase_pop)
            critiqued_matrix = clf.predict(critiqued_matrix_onehot)
            critiqued_matrix_pop = normalize(critiqued_matrix)

            critiqued_matrix_onehot = get_critiqued_UK(U_K,user,critiqued_keyphrase_diff)
            critiqued_matrix = clf.predict(critiqued_matrix_onehot)
            critiqued_matrix_diff = normalize(critiqued_matrix)


#             critiqued_matrix = project_one_hot_encoding(clf, 
#                                                         U_K,
#                                                         user_index = user,
#                                                         critiqued_keyphrase = critiqued_keyphrase, 
#                                                         normalize_en = True)

#             modified_matrix = modified_matrix + critiqued_matrix # averaging user-item embedding and critiquing embeeding
            
            # Warning!!! The following is used only for testing single step critiquing, 
            # for full average critiquing, use the above commented line 
            post_ranks_random_all = []
            post_ranks_random_upper = []
            post_ranks_pop_all = []
            post_ranks_pop_upper = []
            post_ranks_diff_all = []
            post_ranks_diff_upper = []
            
            for lam in lams:
                modified_matrix_random = (1-lam)*normalize(train(matrix_Train)) + lam*critiqued_matrix_random 
                modified_matrix_pop = (1-lam)*normalize(train(matrix_Train)) + lam*critiqued_matrix_pop 
                modified_matrix_diff = (1-lam)*normalize(train(matrix_Train)) + lam*critiqued_matrix_diff 
#                 modified_matrix = normalize(modified_matrix)

            
                # Get new predictions from modified embedding
#                 if recommend_type == 'all':
#                     prediction_scores_u = predict_vector(user, matrix_Train, 100, modified_matrix)
#                 if recommend_type == 'upper':
#                     prediction_scores_u = predict_vector(user, matrix_Train, 100, modified_matrix, 
#                                                          with_keyphrase = True, 
#                                                          keyphrase_freq = keyphrase_freq, 
#                                                          critiqued_keyphrase = critiqued_keyphrase, 
#                                                          alpha = 0)
#                 post_critique_rank = np.where(item == np.argsort(prediction_scores_u)[::-1])[0][0]
#                 print ('target_item post-critique rank with lambda '+str(lam), int(post_critique_rank))
#                 post_rank = int(post_critique_rank)
#                 post_ranks.append(post_rank)
                
                
                # Random 
                prediction_scores_u = predict_vector(user, matrix_Train, 100, modified_matrix_random)
                post_critique_rank = np.where(item == np.argsort(prediction_scores_u)[::-1])[0][0]
                post_ranks_random_all.append(post_critique_rank)
            
                prediction_scores_u = predict_vector(user, matrix_Train, 100, modified_matrix_random, 
                                                         with_keyphrase = True, 
                                                         keyphrase_freq = keyphrase_freq, 
                                                         critiqued_keyphrase = critiqued_keyphrase_random, 
                                                         alpha = 0)
                post_critique_rank = np.where(item == np.argsort(prediction_scores_u)[::-1])[0][0]
                post_ranks_random_upper.append(post_critique_rank)
                
                #Pop
                prediction_scores_u = predict_vector(user, matrix_Train, 100, modified_matrix_pop)
                post_critique_rank = np.where(item == np.argsort(prediction_scores_u)[::-1])[0][0]
                post_ranks_pop_all.append(post_critique_rank)
            
                prediction_scores_u = predict_vector(user, matrix_Train, 100, modified_matrix_pop, 
                                                         with_keyphrase = True, 
                                                         keyphrase_freq = keyphrase_freq, 
                                                         critiqued_keyphrase = critiqued_keyphrase_pop, 
                                                         alpha = 0)
                post_critique_rank = np.where(item == np.argsort(prediction_scores_u)[::-1])[0][0]
                post_ranks_pop_upper.append(post_critique_rank)
                
                #Diff
                prediction_scores_u = predict_vector(user, matrix_Train, 100, modified_matrix_diff)
                post_critique_rank = np.where(item == np.argsort(prediction_scores_u)[::-1])[0][0]
                post_ranks_diff_all.append(post_critique_rank)
            
                prediction_scores_u = predict_vector(user, matrix_Train, 100, modified_matrix_diff, 
                                                         with_keyphrase = True, 
                                                         keyphrase_freq = keyphrase_freq, 
                                                         critiqued_keyphrase = critiqued_keyphrase_diff, 
                                                         alpha = 0)
                post_critique_rank = np.where(item == np.argsort(prediction_scores_u)[::-1])[0][0]
                post_ranks_diff_upper.append(post_critique_rank)
            
            row['post_rank_random_all'] = post_ranks_random_all
            row['post_rank_random_upper'] = post_ranks_random_upper
            row['post_rank_pop_all'] = post_ranks_pop_all
            row['post_rank_pop_upper'] = post_ranks_pop_upper
            row['post_rank_diff_all'] = post_ranks_diff_all
            row['post_rank_diff_upper'] = post_ranks_diff_upper
#             row['post_rank0'] = post_ranks[0]
#             row['post_rank1'] = post_ranks[1]
#             row['post_rank2'] = post_ranks[2]
#             row['post_rank3'] = post_ranks[3]
#             row['post_rank4'] = post_ranks[4]
#             row['post_rank5'] = post_ranks[5]
#             row['post_rank6'] = post_ranks[6]
#             row['post_rank7'] = post_ranks[7]
#             row['post_rank8'] = post_ranks[8]
#             row['post_rank9'] = post_ranks[9]
#             row['post_rank10'] = post_ranks[10]
            df = df.append(row, ignore_index=True)
            

        # break after got 10 target items 
        num_target_item += 1
        if num_target_item >10: # only want max 10 items per user
            break
            
    return df

# Run Experiment

In [38]:
single_step_with_avg_path = "../tables/critiquing/single_step_lam_all/"

In [41]:
# Initialize df for storing the experiment

# post_ranki is post rank with different lambda ratio for combining pre-post User similarity matrix 
columns = ['user_id', 'target_item', 'item_name', 'iter', 'pre_rank', 
           'top_prediction_item_name',
           'post_rank_random_all',
           'post_rank_random_upper',
           'post_rank_pop_all',
           'post_rank_pop_upper',
           'post_rank_diff_all',
           'post_rank_diff_upper',
           'critiqued_keyphrase_random',
           'keyphrase_name_random',
           'critiqued_keyphrase_pop',
           'keyphrase_name_pop',
           'critiqued_keyphrase_diff',
           'keyphrase_name_diff',
           'num_existing_keyphrases'] 
df = pd.DataFrame(columns=columns)
row = {}

#only_with_critiqued_keyphrase
for user in range(300,350):
    df = single_step_critiquing_experiment(user = user, 
                           keyphrase_length_threshold = 230, 
                           max_iteration_threshold = 10,
                           k = 50,
                           df = df,
                           row = row,
                           business_df = business_df,
                           keyphrases = keyphrases,
                           keyphrase_popularity = keyphrase_popularity,
                           keyphrase_selection_method = 'random',
                           recommend_type = 'all'
                           )
df.to_csv(single_step_with_avg_path+"50user.csv")

User ID  300
wanted_items length:  14
Initial top recommendation index 4443
target_item:  101
cur_iter  0
[105]


D:\Anaconda\envs\tensorflow_cpu\lib\site-packages\scipy\sparse\_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
D:\Anaconda\envs\tensorflow_cpu\lib\site-packages\scipy\sparse\_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
D:\Anaconda\envs\tensorflow_cpu\lib\site-packages\scipy\sparse\_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


cur_iter  1
[]
no more keyphrase available
target_item:  886
cur_iter  0
[ 27  48  59  60  72  73 108 115 145 223]


D:\Anaconda\envs\tensorflow_cpu\lib\site-packages\scipy\sparse\_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
D:\Anaconda\envs\tensorflow_cpu\lib\site-packages\scipy\sparse\_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
D:\Anaconda\envs\tensorflow_cpu\lib\site-packages\scipy\sparse\_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


cur_iter  1
[ 27  48  59  60  72  73 108 145 223]


...

D:\Anaconda\envs\tensorflow_cpu\lib\site-packages\scipy\sparse\_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
D:\Anaconda\envs\tensorflow_cpu\lib\site-packages\scipy\sparse\_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
D:\Anaconda\envs\tensorflow_cpu\lib\site-packages\scipy\sparse\_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


cur_iter  8
[ 35  40  50  76 114 164 173 178 203 208]


D:\Anaconda\envs\tensorflow_cpu\lib\site-packages\scipy\sparse\_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
D:\Anaconda\envs\tensorflow_cpu\lib\site-packages\scipy\sparse\_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
D:\Anaconda\envs\tensorflow_cpu\lib\site-packages\scipy\sparse\_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


cur_iter  9
[ 35  40  76 114 164 173 178 203 208]


D:\Anaconda\envs\tensorflow_cpu\lib\site-packages\scipy\sparse\_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
D:\Anaconda\envs\tensorflow_cpu\lib\site-packages\scipy\sparse\_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
D:\Anaconda\envs\tensorflow_cpu\lib\site-packages\scipy\sparse\_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [40]:
df

,user_id,target_item,item_name,iter,pre_rank,top_prediction_item_name,post_rank_random_all,post_rank_random_upper,post_rank_pop_all,post_rank_pop_upper,post_rank_diff_all,post_rank_diff_upper,critiqued_keyphrase_random,keyphrase_name_random,critiqued_keyphrase_pop,keyphrase_name_pop,critiqued_keyphrase_diff,keyphrase_name_diff,num_existing_keyphrases
0,300,101,b'Cabin Fever',0,3311,b'Khao San Road',"[3311, 3536, 2960, 3115, 3546, 3465, 3482, 345...","[2908, 2907, 2903, 2907, 2822, 2812, 2822, 282...","[3311, 3536, 2960, 3115, 3546, 3465, 3482, 345...","[2908, 2907, 2903, 2907, 2822, 2812, 2822, 282...","[3311, 2943, 3521, 3859, 3428, 3601, 3985, 347...","[3113, 3582, 3242, 3502, 3391, 2973, 4389, 260...",105,latte,105,latte,153,friendly,NaN
1,300,886,b'Wok & Roast Chinese BBQ',0,755,b'Khao San Road',"[755, 740, 632, 600, 522, 662, 688, 769, 858, ...","[145, 152, 139, 144, 151, 187, 190, 207, 226, ...","[755, 765, 773, 371, 268, 246, 301, 249, 224, ...","[10, 8, 12, 8, 10, 11, 15, 15, 14, 14, 14]","[755, 795, 473, 424, 445, 873, 811, 815, 830, ...","[237, 245, 172, 164, 182, 304, 287, 278, 285, ...",115,lobster,223,general tao,27,bbq,NaN
2,300,886,b'Wok & Roast Chinese BBQ',1,755,b'Khao San Road',"[755, 784, 783, 756, 885, 1874, 2967, 2934, 29...","[139, 160, 158, 157, 188, 293, 2205, 2264, 223...","[755, 774, 780, 765, 816, 939, 1076, 1842, 202...","[48, 60, 66, 77, 91, 108, 121, 143, 150, 2247,...","[755, 758, 732, 439, 434, 481, 515, 731, 387, ...","[256, 263, 269, 184, 207, 260, 294, 394, 256, ...",60,squid,73,congee,0,chinese,NaN
3,300,886,b'Wok & Roast Chinese BBQ',2,755,b'Khao San Road',"[755, 774, 780, 765, 816, 939, 1076, 1842, 202...","[48, 60, 66, 77, 91, 108, 121, 143, 150, 2247,...","[755, 784, 783, 756, 885, 1874, 2967, 2934, 29...","[139, 160, 158, 157, 188, 293, 2205, 2264, 223...","[755, 772, 795, 882, 583, 575, 2054, 1798, 167...","[379, 385, 396, 447, 340, 350, 883, 804, 781, ...",73,congee,60,squid,51,pork,NaN
4,300,886,b'Wok & Roast Chinese BBQ',3,755,b'Khao San Road',"[755, 830, 928, 2045, 1916, 1962, 4303, 3101, ...","[171, 193, 213, 313, 301, 311, 2240, 2239, 224...","[755, 819, 906, 944, 1364, 2166, 2105, 1946, 1...","[141, 163, 179, 206, 255, 320, 316, 320, 328, ...","[755, 787, 747, 690, 976, 936, 990, 1173, 1484...","[478, 520, 494, 457, 623, 619, 641, 696, 836, ...",108,duck,72,scallop,5,fried,NaN
5,300,886,b'Wok & Roast Chinese BBQ',4,755,b'Khao San Road',"[755, 784, 875, 945, 2129, 2197, 2027, 3059, 4...","[241, 256, 301, 325, 569, 588, 585, 2340, 2329...","[755, 740, 632, 600, 522, 662, 688, 769, 858, ...","[145, 152, 139, 144, 151, 187, 190, 207, 226, ...","[755, 774, 717, 396, 120, 77, 70, 26, 30, 48, 68]","[105, 110, 101, 61, 24, 19, 20, 10, 11, 15, 24]",48,burger,115,lobster,133,fried rice,NaN
6,300,886,b'Wok & Roast Chinese BBQ',5,755,b'Khao San Road',"[755, 865, 1044, 1116, 2032, 3756, 3720, 3065,...","[139, 179, 264, 323, 510, 2321, 2350, 2297, 22...","[755, 830, 928, 2045, 1916, 1962, 4303, 3101, ...","[171, 193, 213, 313, 301, 311, 2240, 2239, 224...","[755, 832, 917, 1065, 1916, 3425, 3392, 3480, ...","[265, 318, 350, 435, 698, 2482, 2480, 2456, 24...",145,markham,108,duck,54,noodle,NaN
7,300,886,b'Wok & Roast Chinese BBQ',6,755,b'Khao San Road',"[755, 819, 906, 944, 1364, 2166, 2105, 1946, 1...","[141, 163, 179, 206, 255, 320, 316, 320, 328, ...","[755, 865, 1044, 1116, 2032, 3756, 3720, 3065,...","[139, 179, 264, 323, 510, 2321, 2350, 2297, 22...","[755, 784, 783, 756, 885, 1874, 2967, 2934, 29...","[139, 160, 158, 157, 188, 293, 2205, 2264, 223...",72,scallop,145,markham,60,squid,NaN
8,300,886,b'Wok & Roast Chinese BBQ',7,755,b'Khao San Road',"[755, 765, 773, 371, 268, 246, 301, 249, 224, ...","[10, 8, 12, 8, 10, 11, 15, 15, 14, 14, 14]","[755, 795, 473, 424, 445, 873, 811, 815, 830, ...","[237, 245, 172, 164, 182, 304, 287, 278, 285, ...","[755, 774, 780, 765, 816, 939, 1076, 1842, 202...","[48, 60, 66, 77, 91, 108, 121, 143, 150, 2247,...",223,general tao,27,bbq,73,congee,NaN
9,300

In [97]:
df

,user_id,target_item,item_name,iter,pre_rank,top_prediction_item_name,critiqued_keyphrase,keyphrase_name,post_rank0,post_rank1,...,post_rank3,post_rank4,post_rank5,post_rank6,post_rank7,post_rank8,post_rank9,post_rank10,num_existing_keyphrases,post_rank
0,300,886,b'Wok & Roast Chinese BBQ',0,755,b'Khao San Road',223,general tao,755,765,...,371,268,246,301,249,224,196,176,10,"[755, 765, 773, 371, 268, 246, 301, 249, 224, ..."
1,300,886,b'Wok & Roast Chinese BBQ',1,755,b'Khao San Road',27,bbq,755,795,...,424,445,873,811,815,830,847,866,10,"[755, 795, 473, 424, 445, 873, 811, 815, 830, ..."
2,300,886,b'Wok & Roast Chinese BBQ',2,755,b'Khao San Road',60,squid,755,784,...,756,885,1874,2967,2934,2901,2919,2903,10,"[755, 784, 783, 756, 885, 1874, 2967, 2934, 29..."
3,300,886,b'Wok & Roast Chinese BBQ',3,755,b'Khao San Road',73,congee,755,774,...,765,816,939,1076,1842,2024,4499,3288,10,"[755, 774, 780, 765, 816, 939, 1076, 1842, 202..."
4,300,886,b'Wok & Roast Chinese BBQ',4,755,b'Khao San Road',72,scallop,755,819,...,944,1364,2166,2105,1946,1955,1933,1923,10,"[755, 819, 906, 944, 1364, 2166, 2105, 1946, 1..."
5,300,886,b'Wok & Roast Chinese BBQ',5,755,b'Khao San Road',108,duck,755,830,...,2045,1916,1962,4303,3101,3083,3054,3098,10,"[755, 830, 928, 2045, 1916, 1962, 4303, 3101, ..."
6,300,886,b'Wok & Roast Chinese BBQ',6,755,b'Khao San Road',48,burger,755,784,...,945,2129,2197,2027,3059,4151,4140,2975,10,"[755, 784, 875, 945, 2129, 2197, 2027, 3059, 4..."
7,300,886,b'Wok & Roast Chinese BBQ',7,755,b'Khao San Road',145,markham,755,865,...,1116,2032,3756,3720,3065,3047,3055,3023,10,"[755, 865, 1044, 1116, 2032, 3756, 3720, 3065,..."
8,300,886,b'Wok & Roast Chinese BBQ',8,755,b'Khao San Road',59,ice cream,755,837,...,1024,1097,1511,1428,1404,1497,1555,2936,10,"[755, 837, 906, 1024, 1097, 1511, 1428, 1404, ..."
9,300,968,b'Porter Airlines',0,574,b'Khao San Road',205,gelato,574,372,...,430,738,1889,3329,3220,3385,2510,3322,10,"[574, 372, 387, 430, 738, 1889, 3329, 3220, 33..."


In [90]:
# pop with normalization
df

,user_id,target_item,item_name,iter,pre_rank,top_prediction_item_name,critiqued_keyphrase,keyphrase_name,post_rank0,post_rank1,...,post_rank3,post_rank4,post_rank5,post_rank6,post_rank7,post_rank8,post_rank9,post_rank10,num_existing_keyphrases,post_rank
0,300,886,b'Wok & Roast Chinese BBQ',0,755,b'Khao San Road',59,ice cream,285,311,...,389,409,471,477,473,505,523,2321,10,"[285, 311, 342, 389, 409, 471, 477, 473, 505, ..."
1,300,886,b'Wok & Roast Chinese BBQ',1,755,b'Khao San Road',73,congee,48,60,...,77,91,108,121,143,150,2247,2247,10,"[48, 60, 66, 77, 91, 108, 121, 143, 150, 2247,..."
2,300,886,b'Wok & Roast Chinese BBQ',2,755,b'Khao San Road',115,lobster,145,152,...,144,151,187,190,207,226,239,247,10,"[145, 152, 139, 144, 151, 187, 190, 207, 226, ..."
3,300,886,b'Wok & Roast Chinese BBQ',3,755,b'Khao San Road',223,general tao,10,8,...,8,10,11,15,15,14,14,14,10,"[10, 8, 12, 8, 10, 11, 15, 15, 14, 14, 14]"
4,300,886,b'Wok & Roast Chinese BBQ',4,755,b'Khao San Road',60,squid,139,160,...,157,188,293,2205,2264,2235,2237,2235,10,"[139, 160, 158, 157, 188, 293, 2205, 2264, 223..."
5,300,968,b'Porter Airlines',0,574,b'Khao San Road',42,sandwich,267,279,...,318,292,308,406,380,371,368,214,10,"[267, 279, 296, 318, 292, 308, 406, 380, 371, ..."
6,300,968,b'Porter Airlines',1,574,b'Khao San Road',103,fruit,255,185,...,170,148,36,24,20,16,24,23,10,"[255, 185, 199, 170, 148, 36, 24, 20, 16, 24, 23]"
7,300,968,b'Porter Airlines',2,574,b'Khao San Road',38,chocolate,273,287,...,268,231,183,351,331,322,297,282,10,"[273, 287, 300, 268, 231, 183, 351, 331, 322, ..."
8,300,968,b'Porter Airlines',3,574,b'Khao San Road',230,alcoholic beverage,25,30,...,27,33,33,26,23,19,16,17,10,"[25, 30, 23, 27, 33, 33, 26, 23, 19, 16, 17]"
9,300,968,b'Porter Airlines',4,574,b'Khao San Road',65,espresso,117,84,...,38,41,67,48,49,43,39,36,10,"[117, 84, 66, 38, 41, 67, 48, 49, 43, 39, 36]"


In [93]:
# without normalization
df

,user_id,target_item,item_name,iter,pre_rank,top_prediction_item_name,critiqued_keyphrase,keyphrase_name,post_rank0,post_rank1,...,post_rank3,post_rank4,post_rank5,post_rank6,post_rank7,post_rank8,post_rank9,post_rank10,num_existing_keyphrases,post_rank
0,300,886,b'Wok & Roast Chinese BBQ',0,755,b'Khao San Road',59,ice cream,285,311,...,389,409,471,477,473,505,523,2321,10,"[285, 311, 342, 389, 409, 471, 477, 473, 505, ..."
1,300,886,b'Wok & Roast Chinese BBQ',1,755,b'Khao San Road',223,general tao,10,8,...,8,10,11,15,15,14,14,14,10,"[10, 8, 12, 8, 10, 11, 15, 15, 14, 14, 14]"
2,300,886,b'Wok & Roast Chinese BBQ',2,755,b'Khao San Road',145,markham,139,179,...,323,510,2321,2350,2297,2298,2298,2295,10,"[139, 179, 264, 323, 510, 2321, 2350, 2297, 22..."
3,300,886,b'Wok & Roast Chinese BBQ',3,755,b'Khao San Road',60,squid,139,160,...,157,188,293,2205,2264,2235,2237,2235,10,"[139, 160, 158, 157, 188, 293, 2205, 2264, 223..."
4,300,886,b'Wok & Roast Chinese BBQ',4,755,b'Khao San Road',72,scallop,141,163,...,206,255,320,316,320,328,328,332,10,"[141, 163, 179, 206, 255, 320, 316, 320, 328, ..."
5,300,968,b'Porter Airlines',0,574,b'Khao San Road',38,chocolate,273,287,...,268,231,183,351,331,322,297,282,10,"[273, 287, 300, 268, 231, 183, 351, 331, 322, ..."
6,300,968,b'Porter Airlines',1,574,b'Khao San Road',105,latte,335,333,...,333,305,307,330,182,183,220,219,10,"[335, 333, 331, 333, 305, 307, 330, 182, 183, ..."
7,300,968,b'Porter Airlines',2,574,b'Khao San Road',69,cookie,115,90,...,112,202,295,289,279,3508,3507,3505,10,"[115, 90, 98, 112, 202, 295, 289, 279, 3508, 3..."
8,300,968,b'Porter Airlines',3,574,b'Khao San Road',230,alcoholic beverage,25,30,...,27,33,33,26,23,19,16,17,10,"[25, 30, 23, 27, 33, 33, 26, 23, 19, 16, 17]"
9,300,968,b'Porter Airlines',4,574,b'Khao San Road',143,station,200,140,...,102,75,82,70,68,79,75,74,10,"[200, 140, 145, 102, 75, 82, 70, 68, 79, 75, 74]"


# Visualization 

### Plot HR vs. Performance for 3 keyphrase selection method

In [66]:
# Utlilty func
def hr_at_k(df, k):
    """
    Given the above dataframe, calculate the avg pre and post hit rate at k 
    """
    pre_hit = np.where(df['pre_rank']<k)[0]
    post_hit = np.where(df['post_rank']<k)[0]
    pre_hr = len(pre_hit)/len(df)
    post_hr = len(post_hit)/len(df)
    return pre_hr, post_hr

def get_hr(l=5, rang = 200):
    """
    Get the hit rate at different rang, with different lambda value
    Output in the form of list
    """
    pre_hr_list = []
    post_hr_list = []
    for k in range(1,rang):
        pre_hr,post_hr = hr_at_k(df,4, k)
        pre_hr_list.append(pre_hr)
        post_hr_list.append(post_hr)
    return pre_hr_list,post_hr_list

def get_hr_of_all_lambda(methods):
    for method in methods:
        post_rates = []
        for i in range(9):
            _,a = get_hr(l=i)
            diff.append(a)

In [ ]:
pre_hr_list,b = get_hr(l=5)

In [ ]:
def plot_hr_performance():
    plt.figure(figsize=(12,12))
    for 
    plt.plot(np.arange(len(pre_hr_list)), pre_hr_list)
    plt.plot(np.arange(len(pre_hr_list)), a1)
    plt.plot(np.arange(len(pre_hr_list)), a2)
    plt.plot(np.arange(len(pre_hr_list)), a3)
    plt.plot(np.arange(len(pre_hr_list)), a4)
    plt.plot(np.arange(len(pre_hr_list)), a5)
    plt.plot(np.arange(len(pre_hr_list)), a6)

    # plt.xticks(np.arange(len(k_list)), k_list)
    plt.xlabel('k')
    plt.ylabel('HR@K')
    plt.legend(['Pre-Critiquing','Random', 'Random_Upper','Pop','Pop_Upper','Diff','Diff_Upper'])
    plt.show()
    plt.savefig('../figs/three_keyphrase_selection_methods_with_upper_bound_0104')